In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

test_dates = data["date"].iloc[len(ytr):len(ytr)+len(yte)]

plt.figure(figsize=(11,4))
plt.plot(test_dates, yte.values, label="Réel (discharges)")
plt.plot(test_dates, pred_te, label="Prédit (modèle)", linewidth=2)
plt.title("Test : discharges — Réel vs Prédit"); plt.xlabel(""); plt.ylabel("lits/jour")
plt.legend(); plt.tight_layout(); plt.show()

resid = yte.values - pred_te
print(pd.Series(resid).describe().round(2))

plt.figure(figsize=(11,2.8))
plt.plot(test_dates, resid); plt.axhline(0, ls="--")
plt.title("Résidus (réel - prédit)"); plt.xlabel(""); plt.tight_layout(); plt.show()


In [ ]:
import joblib, os, pandas as pd
from datetime import timedelta

os.makedirs("../models", exist_ok=True)
joblib.dump(m, "../models/baseline_linreg.pkl")

last_date = df["date"].max()
d1, d2 = last_date + pd.Timedelta(days=1), last_date + pd.Timedelta(days=2)

# Reprends p1, p2 que tu as déjà calculés
pred_out = pd.DataFrame({"date":[d1.date(), d2.date()],
                         "pred_discharges":[round(p1), round(p2)]})
os.makedirs("../data/pred", exist_ok=True)
pred_out.to_csv("../data/pred/discharges_J1_J2.csv", index=False)
pred_out


In [ ]:
import pandas as pd, joblib
import streamlit as st
from pathlib import Path
from datetime import timedelta

st.set_page_config(page_title="SPH - Prévision sorties", layout="wide")

root = Path(__file__).parents[1]
df = pd.read_csv(root/"data/raw/hospital_daily.csv", parse_dates=["date"]).sort_values("date")
m  = joblib.load(root/"models/baseline_linreg.pkl")

# Features pour une date cible
def make_feats(date_target, hist):
    s = hist.set_index("date")["discharges"]
    feats = {
        "dow": date_target.weekday(),
        "lag1": s.iloc[-1],
        "lag7": s.iloc[-7],
        "ma7":  s.iloc[-7:].mean()
    }
    return pd.DataFrame([feats], columns=["dow","lag1","lag7","ma7"])

last = df["date"].max()
d1, d2 = last + pd.Timedelta(days=1), last + pd.Timedelta(days=2)
p1 = float(m.predict(make_feats(d1, df))[0])
# pour J+2 on réutilise p1
s = df.set_index("date")["discharges"]
lag7_d2 = s.iloc[-6]; ma7_d2 = pd.Series([*s.iloc[-6:], p1]).mean()
X2 = pd.DataFrame([{"dow": d2.weekday(), "lag1": p1, "lag7": lag7_d2, "ma7": ma7_d2}],
                  columns=["dow","lag1","lag7","ma7"])
p2 = float(m.predict(X2)[0])

st.title("Pilotage SPH : prévisions de sorties")
c1, c2 = st.columns(2)
c1.metric(f"J+1 ({d1.date()})", f"{round(p1)} lits")
c2.metric(f"J+2 ({d2.date()})", f"{round(p2)} lits")

st.line_chart(df.set_index("date")["discharges"])
st.caption("Sorties = proxy de charge ménage. Modèle baseline (lags + MA7).")


In [ ]:
threshold = int(df["discharges"].quantile(0.75))  # 75e percentile historique
alert_j1 = "🔴 Alerte" if p1 >= threshold else "🟢 OK"
alert_j2 = "🔴 Alerte" if p2 >= threshold else "🟢 OK"
st.write(f"Seuil alerte: {threshold} | J+1: {alert_j1} | J+2: {alert_j2}")
